In [ ]:
using GeoArrays
using DataFrames
using MLJ
using Plots

In [ ]:
s2 = GeoArrays.read("../../sentinel2.tif")

In [ ]:
df = reshape(s2, (size(s2)[1]*size(s2)[2], size(s2)[3])) |>
x -> DataFrame(x, :auto)

In [ ]:
df = coerce(df, Count=>Continuous)

In [ ]:
# K-Means
model_kmeans = @load KMeans pkg=Clustering
pipe_kmeans = @pipeline Standardizer() model_kmeans(k=10)
mach_kmeans = machine(pipe_kmeans, df) |> fit!;

In [ ]:
rep_kmeans = report(mach_kmeans)

In [ ]:
df[!, :clusters] = collect(rep_kmeans.k_means[1]);

In [ ]:
kmeans_raster = reshape(Array(df), (size(s2)[1], size(s2)[2], size(s2)[3]+1)) |>
x -> reverse(x, dims=2) |>
GeoArray

In [ ]:
GeoArrays.bbox!(kmeans_raster, GeoArrays.bbox(s2))
GeoArrays.epsg!(kmeans_raster, 32610)
GeoArrays.write!("../../sentinel2_kmeans.tif", kmeans_raster)

In [ ]:
p1 = plot(s2, band=7)
p2 = plot(kmeans_raster, band=14)
plot(p1, p2, layout=(2,1), size=(800,800))